In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader

import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0.5, 0.5)])
train_dataset = datasets.MNIST(root='/data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='/data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, drop_last=True)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop = nn.Dropout(p=0.9)
        self.conv1 = nn.Conv2d(1, 6, 5)         # (28x28) -> (24x24)
        self.conv2 = nn.Conv2d(6, 16, 5)        # (24x24) -> (20x20)
        self.fc2 = nn.Linear(16*20*20, 10)

    def forward(self, x):
        x = F.relu(self.drop(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = self.fc2(x.view(-1, 16*20*20))
        return x

In [26]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = Net().to(device)

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-5, betas=(0.99, 0.999))

In [38]:
epochs = 10
for epoch in range(epochs):
    for _, samples in enumerate(train_loader):
        images, labels = samples
        images = images.to(device)
        labels = labels.to(device)

        output = net(images)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('epoch: {:2d}/{}, \t loss: {:.3f}'.format(epoch+1, epochs, loss))

epoch:  1/10, 	 loss: 0.274
epoch:  2/10, 	 loss: 0.102
epoch:  3/10, 	 loss: 0.110
epoch:  4/10, 	 loss: 0.108
epoch:  5/10, 	 loss: 0.178
epoch:  6/10, 	 loss: 0.157
epoch:  7/10, 	 loss: 0.157
epoch:  8/10, 	 loss: 0.274
epoch:  9/10, 	 loss: 0.115
epoch: 10/10, 	 loss: 0.195


In [39]:
correct = 0
total = 0

for _, samples in enumerate(test_loader):
    images, labels = samples
    images = images.to(device)
    labels = labels.to(device)

    output = net(images)
    predictions = output.argmax(dim=1)

    correct += (predictions == labels).sum()
    total += labels.shape[0]

print(f'accuracy: {correct / total * 100:.3f}%')

    

accuracy: 94.772%


In [34]:
class Resnet(nn.Module):

    def __init__(self):
        super(Resnet, self).__init__()
        # first layer
        self.fc1 = nn.Linear(1*28*28, 1*28*28)
        # residual layer
        self.conv1 = nn.Conv2d(1, 1, 5, padding=2)
        # last layer
        self.fc2 = nn.Linear(1*28*28, 10)

    def forward(self, x):
        x_shape = x.shape
        x = F.relu(self.fc1(x.view(-1, 1*28*28))).view(x_shape)

        # resnet block
        x = F.relu(self.conv1(x)) + x

        x = self.fc2(x.view(-1, 1*28*28))       # vectorizing x to forward to the linear layer
        return x

net = Resnet().to(device)
t = torch.randn((64, 1, 28, 28)).to(device)
net(t).shape

torch.Size([64, 10])